In [1]:
import pandas as pd
import json

In [5]:
 data=[
    {
        "id": 1,
        "number_of_guest": 2,
        "pickup.datetime": "2023-07-01T12:00:00Z",
        "dropoff.datetime": "2023-07-01T12:30:00Z",
        "pickup.longitude": -73.9857,
        "pickup.latitude": 40.7484,
        "dropoff.longitude": -73.9857,
        "dropoff.latitude": 40.7484,
        "type": "Standard",
        "trip_distance": 5.2,
        "fare_amount": 15.5,
        
    },
    {
        "id": 2,
        "number_of_guest": 1,
        "pickup.datetime": "2023-07-01T13:00:00Z",
        "dropoff.datetime": "2023-07-01T13:20:00Z",
        "pickup.longitude": -73.9757,
        "pickup.latitude": 40.7584,
        "dropoff.longitude": -73.9757,
        "dropoff.latitude": 40.7584,
        "type": "Premium",
        "trip_distance": 3.4,
        "fare_amount": 12.0,
        
    }
]


In [6]:
df=pd.json_normalize(data)

In [4]:
df

,id,number_of_guest,pickup.datetime,dropoff.datetime,pickup.longitude,pickup.latitude,dropoff.longitude,dropoff.latitude,type,trip_distance,fare_amount,tip_amount,total_amount
0,1,2,2023-07-01T12:00:00Z,2023-07-01T12:30:00Z,-73.9857,40.7484,-73.9857,40.7484,Standard,5.2,15.5,2.5,18.0
1,2,1,2023-07-01T13:00:00Z,2023-07-01T13:20:00Z,-73.9757,40.7584,-73.9757,40.7584,Premium,3.4,12.0,3.0,15.0


In [7]:
df_melted = df.melt(id_vars=['id', 'number_of_guest', 'type', 'trip_distance'], 
                    value_vars=['pickup.datetime', 'dropoff.datetime', 'pickup.longitude', 'pickup.latitude', 'dropoff.longitude', 'dropoff.latitude', 'fare_amount'],
                    var_name='variable', 
                    value_name='value')

In [8]:
df_melted

,id,number_of_guest,type,trip_distance,variable,value
0,1,2,Standard,5.2,pickup.datetime,2023-07-01T12:00:00Z
1,2,1,Premium,3.4,pickup.datetime,2023-07-01T13:00:00Z
2,1,2,Standard,5.2,dropoff.datetime,2023-07-01T12:30:00Z
3,2,1,Premium,3.4,dropoff.datetime,2023-07-01T13:20:00Z
4,1,2,Standard,5.2,pickup.longitude,-73.9857
5,2,1,Premium,3.4,pickup.longitude,-73.9757
6,1,2,Standard,5.2,pickup.latitude,40.7484
7,2,1,Premium,3.4,pickup.latitude,40.7584
8,1,2,Standard,5.2,dropoff.longitude,-73.9857
9,2,1,Premium,3.4,dropoff.longitude,-73.9757


In [9]:
df_melted[['location', 'metric']] = df_melted['variable'].str.split('.', expand=True)

In [10]:
df_melted

,id,number_of_guest,type,trip_distance,variable,value,location,metric
0,1,2,Standard,5.2,pickup.datetime,2023-07-01T12:00:00Z,pickup,datetime
1,2,1,Premium,3.4,pickup.datetime,2023-07-01T13:00:00Z,pickup,datetime
2,1,2,Standard,5.2,dropoff.datetime,2023-07-01T12:30:00Z,dropoff,datetime
3,2,1,Premium,3.4,dropoff.datetime,2023-07-01T13:20:00Z,dropoff,datetime
4,1,2,Standard,5.2,pickup.longitude,-73.9857,pickup,longitude
5,2,1,Premium,3.4,pickup.longitude,-73.9757,pickup,longitude
6,1,2,Standard,5.2,pickup.latitude,40.7484,pickup,latitude
7,2,1,Premium,3.4,pickup.latitude,40.7584,pickup,latitude
8,1,2,Standard,5.2,dropoff.longitude,-73.9857,dropoff,longitude
9,2,1,Premium,3.4,dropoff.longitude,-73.9757,dropoff,longitude


In [11]:
df_tidy = df_melted.pivot_table(index=['id', 'number_of_guest', 'type', 'trip_distance', 'location'], 
                                columns='metric', 
                                values='value', 
                                aggfunc='first').reset_index()

In [12]:
df_tidy

metric,id,number_of_guest,type,trip_distance,location,datetime,latitude,longitude
0,1,2,Standard,5.2,dropoff,2023-07-01T12:30:00Z,40.7484,-73.9857
1,1,2,Standard,5.2,pickup,2023-07-01T12:00:00Z,40.7484,-73.9857
2,2,1,Premium,3.4,dropoff,2023-07-01T13:20:00Z,40.7584,-73.9757
3,2,1,Premium,3.4,pickup,2023-07-01T13:00:00Z,40.7584,-73.9757
